In [1]:
# Cell 1 — write scene.xml (adds the visible green goal marker site: push_goal)

scene_xml = """<mujoco model="so_arm100 scene">
  <include file="so_arm100.xml"/>

  <statistic center="0.1 -0.01 0.05" extent="0.5"/>

  <visual>
    <headlight diffuse="0.6 0.6 0.6" ambient="0.3 0.3 0.3" specular="0 0 0"/>
    <rgba haze="0.15 0.25 0.35 1"/>
    <global azimuth="45" elevation="-20"/>
  </visual>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="512" height="3072"/>
    <texture type="2d" name="groundplane" builtin="checker" mark="edge" rgb1="0.2 0.3 0.4" rgb2="0.1 0.2 0.3"
      markrgb="0.8 0.8 0.8" width="300" height="300"/>
    <material name="groundplane" texture="groundplane" texuniform="true" texrepeat="5 5" reflectance="0.2"/>
  </asset>

  <worldbody>
    <light pos="0 0 1.5" dir="0 0 -1" directional="true"/>
    <geom name="floor" size="0 0 0.05" type="plane" material="groundplane"/>
    <site name="target" pos="0.2 -0.2 0.05" size="0.03" rgba="1 0 0 0.5" type="sphere"/>
    <!-- Visual marker for the PUSH task goal (updated at reset by the env) -->
    <site name="push_goal" pos="0.0 0.0 0.02" size="0.03" rgba="0 1 0 0.5" type="sphere"/>
  </worldbody>
</mujoco>
"""

with open("scene.xml", "w", encoding="utf-8") as f:
    f.write(scene_xml)

print("Wrote scene.xml")


Wrote scene.xml


In [2]:
# Cell 2 — write so_arm100.xml (fixed cube geom line; includes ee_site + fixed_jaw_site)

so_arm100_xml = """<mujoco model="so_arm100">
  <compiler angle="radian" meshdir="assets"/>

  <option cone="elliptic" impratio="10"/>

  <asset>
    <material name="white" rgba="1 1 1 1"/>
    <material name="black" rgba="0.1 0.1 0.1 1"/>

    <mesh name="Base" file="Base.stl"/>
    <mesh name="Base_Motor" file="Base_Motor.stl"/>
    <mesh name="Rotation_Pitch" file="Rotation_Pitch.stl"/>
    <mesh name="Rotation_Pitch_Motor" file="Rotation_Pitch_Motor.stl"/>
    <mesh name="Upper_Arm" file="Upper_Arm.stl"/>
    <mesh name="Upper_Arm_Motor" file="Upper_Arm_Motor.stl"/>
    <mesh name="Lower_Arm" file="Lower_Arm.stl"/>
    <mesh name="Lower_Arm_Motor" file="Lower_Arm_Motor.stl"/>
    <mesh name="Wrist_Pitch_Roll" file="Wrist_Pitch_Roll.stl"/>
    <mesh name="Wrist_Pitch_Roll_Motor" file="Wrist_Pitch_Roll_Motor.stl"/>
    <mesh name="Fixed_Jaw" file="Fixed_Jaw.stl"/>
    <mesh name="Fixed_Jaw_Motor" file="Fixed_Jaw_Motor.stl"/>
    <mesh name="Fixed_Jaw_Collision_1" file="Fixed_Jaw_Collision_1.stl"/>
    <mesh name="Fixed_Jaw_Collision_2" file="Fixed_Jaw_Collision_2.stl"/>
    <mesh name="Moving_Jaw" file="Moving_Jaw.stl"/>
    <mesh name="Moving_Jaw_Collision_1" file="Moving_Jaw_Collision_1.stl"/>
    <mesh name="Moving_Jaw_Collision_2" file="Moving_Jaw_Collision_2.stl"/>
    <mesh name="Moving_Jaw_Collision_3" file="Moving_Jaw_Collision_3.stl"/>
  </asset>

  <default>
    <default class="so_arm100">
      <joint frictionloss="0.1" armature="0.1"/>
      <position kp="50" dampratio="1" forcerange="-3.5 3.5"/>

      <default class="Rotation">
        <joint axis="0 1 0" range="-1.92 1.92" ref="0"/>
      </default>

      <default class="Pitch">
        <joint axis="1 0 0" range="-3.32 0.174" ref="0"/>
      </default>

      <default class="Elbow">
        <joint axis="1 0 0" range="-0.174 3.14" ref="0"/>
      </default>

      <default class="Wrist_Pitch">
        <joint axis="1 0 0" range="-1.66 1.66" ref="0"/>
      </default>

      <default class="Wrist_Roll">
        <joint axis="0 1 0" range="-2.79 2.79" ref="0"/>
      </default>

      <default class="Jaw">
        <joint axis="0 0 1" range="-0.174 1.75" ref="0"/>
      </default>

      <default class="visual">
        <geom type="mesh" contype="0" conaffinity="0" density="0" group="2" material="white"/>
        <default class="motor_visual">
          <geom material="black"/>
        </default>
      </default>

      <default class="collision">
        <geom group="3" type="mesh" material="white"/>
        <default class="finger_collision">
          <geom type="box" solimp="2 1 0.01" solref="0.01 1" friction="2.5 0.005 0.0001"/>
        </default>
      </default>
    </default>
  </default>

  <worldbody>

    <!-- === Small cube for the task === -->
    <body name="cube" pos="0.25 0.10 0.02" >
      <!-- size: 2cm x 2cm x 2cm -->
      <joint name="cube" type="free" damping="0.01"/> 
      <geom name="cube_geom" type="box" size="0.02 0.02 0.02" rgba="0 1 0 1" density="2000" condim="4" friction="1 0.005 0.0001"/>
    </body>

    <!-- === Robot base and kinematic chain === -->
    <body name="Base" childclass="so_arm100" euler="0 0 1.57075">
      <geom type="mesh" mesh="Base" class="visual"/>
      <geom type="mesh" mesh="Base_Motor" class="motor_visual"/>
      <geom type="mesh" mesh="Base" class="collision"/>

      <body name="Rotation_Pitch" pos="0 -0.0452 0.0165" quat="0.707105 0.707108 0 0">
        <inertial pos="-9.07886e-05 0.0590972 0.031089"
                  quat="0.363978 0.441169 -0.623108 0.533504"
                  mass="0.119226"
                  diaginertia="5.94278e-05 5.89975e-05 3.13712e-05"/>
        <joint name="Rotation" class="Rotation"/>
        <geom type="mesh" mesh="Rotation_Pitch" class="visual"/>
        <geom type="mesh" mesh="Rotation_Pitch_Motor" class="motor_visual"/>
        <geom type="mesh" mesh="Rotation_Pitch" class="collision"/>

        <body name="Upper_Arm" pos="0 0.1025 0.0306" euler="1.57079 0 0">
          <inertial pos="-1.72052e-05 0.0701802 0.00310545"
                    quat="0.50104 0.498994 -0.493562 0.50632"
                    mass="0.162409"
                    diaginertia="0.000213312 0.000167164 7.01522e-05"/>
          <joint name="Pitch" class="Pitch"/>
          <geom type="mesh" mesh="Upper_Arm" class="visual"/>
          <geom type="mesh" mesh="Upper_Arm_Motor" class="motor_visual"/>
          <geom type="mesh" mesh="Upper_Arm" class="collision"/>

          <body name="Lower_Arm" pos="0 0.11257 0.028" euler="-1.57079 0 0">
            <inertial pos="-0.00339604 0.00137796 0.0768007"
                      quat="0.701995 0.0787996 0.0645626 0.704859"
                      mass="0.147968"
                      diaginertia="0.000138803 0.000107748 4.84242e-05"/>
            <joint name="Elbow" class="Elbow"/>
            <geom type="mesh" mesh="Lower_Arm" class="visual"/>
            <geom type="mesh" mesh="Lower_Arm_Motor" class="motor_visual"/>
            <geom type="mesh" mesh="Lower_Arm" class="collision"/>

            <body name="Wrist_Pitch_Roll" pos="0 0.0052 0.1349" euler="-1.57079 0 0">
              <inertial pos="-0.00852653 -0.0352279 -2.34622e-05"
                        quat="-0.0522806 0.705235 0.0549524 0.704905"
                        mass="0.0661321"
                        diaginertia="3.45403e-05 2.39041e-05 1.94704e-05"/>
              <joint name="Wrist_Pitch" class="Wrist_Pitch"/>
              <geom type="mesh" mesh="Wrist_Pitch_Roll" class="visual"/>
              <geom type="mesh" mesh="Wrist_Pitch_Roll_Motor" class="motor_visual"/>
              <geom type="mesh" mesh="Wrist_Pitch_Roll" class="collision"/>

              <body name="Fixed_Jaw" pos="0 -0.0601 0" euler="0 1.57079 0">
                <inertial pos="0.00552377 -0.0280167 0.000483583"
                          quat="0.41836 0.620891 -0.350644 0.562599"
                          mass="0.0929859"
                          diaginertia="5.03136e-05 4.64098e-05 2.72961e-05"/>
                <joint name="Wrist_Roll" class="Wrist_Roll"/>

                <geom type="mesh" mesh="Fixed_Jaw" class="visual"/>
                <geom type="mesh" mesh="Fixed_Jaw_Motor" class="motor_visual"/>

                <!-- NEW SITE: Tip of the Fixed Jaw -->
                <site name="fixed_jaw_site" type="sphere" pos="0.008 -0.095 0" size="0.005" rgba="0 0 1 1"/>
                
                <geom type="mesh" mesh="Fixed_Jaw_Collision_1" class="collision"/>
                <geom type="mesh" mesh="Fixed_Jaw_Collision_2" class="collision"/>

                <geom class="finger_collision" name="fixed_jaw_pad_1" size="0.001 0.005 0.004" pos="0.0089 -0.1014 0"/>
                <geom class="finger_collision" name="fixed_jaw_pad_2" size="0.001 0.005 0.006" pos="0.0109 -0.0914 0"/>
                <geom class="finger_collision" name="fixed_jaw_pad_3" size="0.001 0.01 0.007" pos="0.0126 -0.0768 0"/>
                <geom class="finger_collision" name="fixed_jaw_pad_4" size="0.001 0.01 0.008" pos="0.0143 -0.0572 0"/>

                <!-- ============== MOVING JAW ============== -->
                <body name="Moving_Jaw" pos="-0.0202 -0.0244 0"
                      quat="1.34924e-11 -3.67321e-06 1 -3.67321e-06">

                  <inertial pos="-0.00161745 -0.0303473 0.000449646"
                            quat="0.696562 0.716737 -0.0239844 -0.0227026"
                            mass="0.0202444"
                            diaginertia="1.11265e-05 8.99651e-06 2.99548e-06"/>

                  <joint name="Jaw" class="Jaw"/>

                  <geom type="mesh" mesh="Moving_Jaw" class="visual"/>
                  <geom type="mesh" mesh="Moving_Jaw_Collision_1" class="collision"/>
                  <geom type="mesh" mesh="Moving_Jaw_Collision_2" class="collision"/>
                  <geom type="mesh" mesh="Moving_Jaw_Collision_3" class="collision"/>

                  <geom class="finger_collision" name="moving_jaw_pad_1" size="0.001 0.005 0.004" pos="-0.0113 -0.077 0"/>
                  <geom class="finger_collision" name="moving_jaw_pad_2" size="0.001 0.005 0.006" pos="-0.0093 -0.067 0"/>
                  <geom class="finger_collision" name="moving_jaw_pad_3" size="0.001 0.01 0.006" pos="-0.0073 -0.055 0"/>
                  <geom class="finger_collision" name="moving_jaw_pad_4" size="0.001 0.01 0.008" pos="-0.0073 -0.035 0"/>

                  <!-- === End-effector site on gripper tip === -->
                  <site name="ee_site"
                        type="sphere"
                        pos="-0.012 -0.070 0"
                        size="0.005"
                        rgba="1 0 0 1"/>

                </body> <!-- end Moving_Jaw -->

              </body> <!-- end Fixed_Jaw -->
            </body>   <!-- end Wrist_Pitch_Roll -->
          </body>     <!-- end Lower_Arm -->
        </body>       <!-- end Upper_Arm -->
      </body>         <!-- end Rotation_Pitch -->
    </body>           <!-- end Base -->
  </worldbody>

  <actuator>
   <position class="Rotation" name="Rotation" joint="Rotation" inheritrange="1"/>
   <position class="Pitch" name="Pitch" joint="Pitch" inheritrange="1"/>
   <position class="Elbow" name="Elbow" joint="Elbow" inheritrange="1"/>
   <position class="Wrist_Pitch" name="Wrist_Pitch" joint="Wrist_Pitch" inheritrange="1"/>
   <position class="Wrist_Roll" name="Wrist_Roll" joint="Wrist_Roll" inheritrange="1"/> 
   <position class="Jaw" name="Jaw" joint="Jaw" inheritrange="1"/>
  </actuator>

  <contact>
    <exclude body1="Base" body2="Rotation_Pitch"/>
  </contact>

  <keyframe>
    <key name="home" qpos="0.25 0.10 0.2 1 0 0 0 0 -1.57 1.57 1.57 1.57 0" ctrl="0 -1.57 1.57 1.57 1.57 0"/>
    <key name="rest" qpos="0 0 0 1 0 0 0 0 -3.32 3.11 1.18 0 -0.174" ctrl="0 -3.32 3.11 1.18 0 -0.174"/>
  </keyframe>
</mujoco>
"""

with open("so_arm100.xml", "w", encoding="utf-8") as f:
    f.write(so_arm100_xml)

print("Wrote so_arm100.xml")


Wrote so_arm100.xml


In [3]:
# Cell 3 — write so_arm_simple_env.py (low-level Mujoco wrapper, 18D obs + delta position control)

so_arm_simple_env_py = r'''"""Low-level MuJoCo wrapper used by the high-level Gym environments.

This repo originally referenced a `so_arm_simple_env.py` that wasn't included
in the uploaded files. This implementation is intentionally minimal but
feature-complete for PPO/DDPG style training with Stable-Baselines3.

Key features:
  - Loads MuJoCo XML (scene.xml that includes so_arm100.xml).
  - Position-control actuators: actions are joint *deltas* scaled by
    `action_scale` and applied to `data.ctrl`.
  - Fixed control rate (control_hz). Internally performs multiple mujoco
    simulation steps per control step.
  - Provides a standard 18D observation used by your envs:
      [q(6), dq(6), cube_pos(3), ee_pos(3)]
  - Exposes `model`, `data`, `sim_dt`, and `ctrl_steps` for visualization.

Assumptions (based on the provided XML):
  - Robot joints are named: Rotation, Pitch, Elbow, Wrist_Pitch, Wrist_Roll, Jaw
  - Cube body is named: cube (geom: cube_geom)
  - End-effector site is named: ee_site
"""

from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable, List, Optional, Tuple

import numpy as np
import mujoco


@dataclass
class JointSpec:
    name: str
    joint_id: int
    qpos_adr: int
    dof_adr: int
    range_min: float
    range_max: float


class SoArmSimpleEnv:
    def __init__(
        self,
        xml_path: str = "scene.xml",
        control_hz: int = 20,
        episode_seconds: float = 10.0,
        action_scale: float = 0.05,
        joint_names: Optional[Iterable[str]] = None,
        cube_body: str = "cube",
        ee_site: str = "ee_site",
        cube_init_xy: Tuple[float, float] = (0.25, 0.10),
        cube_init_z: float = 0.02,
        cube_xy_noise: float = 0.02,
    ):
        self.model = mujoco.MjModel.from_xml_path(xml_path)
        self.data = mujoco.MjData(self.model)

        self.control_hz = int(control_hz)
        self.episode_seconds = float(episode_seconds)
        self.action_scale = float(action_scale)

        self.sim_dt = float(self.model.opt.timestep)
        self.ctrl_steps = max(1, int(round((1.0 / self.control_hz) / self.sim_dt)))
        self.max_steps = int(round(self.episode_seconds * self.control_hz))

        # --- IDs ---
        self.cube_body_id = self.model.body(cube_body).id
        self.ee_site_id = self.model.site(ee_site).id

        # --- Actuated joints ---
        if joint_names is None:
            joint_names = ["Rotation", "Pitch", "Elbow", "Wrist_Pitch", "Wrist_Roll", "Jaw"]
        self.joints: List[JointSpec] = []
        for name in joint_names:
            j = self.model.joint(name)
            jid = j.id
            qpos_adr = int(self.model.jnt_qposadr[jid])
            dof_adr = int(self.model.jnt_dofadr[jid])
            rmin, rmax = self.model.jnt_range[jid]
            self.joints.append(JointSpec(name, jid, qpos_adr, dof_adr, float(rmin), float(rmax)))

        self.n_joints = len(self.joints)

        # Observation: q(6)+dq(6)+cube_pos(3)+ee_pos(3)
        self.n_obs = self.n_joints * 2 + 6

        # --- Reset defaults ---
        self._home_q = np.array([0.0, -1.57, 1.57, 1.57, 1.57, 0.0], dtype=np.float64)
        self._cube_init_xy = np.array(cube_init_xy, dtype=np.float64)
        self._cube_init_z = float(cube_init_z)
        self._cube_xy_noise = float(cube_xy_noise)

        # Precompute cube free-joint qpos address (7 values for free joint)
        self._cube_joint_qpos_adr = int(self.model.joint("cube").qposadr)

        self._rng = np.random.default_rng()

        # Make sure state is valid
        self.reset()

    # ---------------------------------------------------------------------
    # Utilities
    # ---------------------------------------------------------------------
    def _get_q(self) -> np.ndarray:
        q = np.empty(self.n_joints, dtype=np.float64)
        for i, js in enumerate(self.joints):
            q[i] = self.data.qpos[js.qpos_adr]
        return q

    def _get_dq(self) -> np.ndarray:
        dq = np.empty(self.n_joints, dtype=np.float64)
        for i, js in enumerate(self.joints):
            dq[i] = self.data.qvel[js.dof_adr]
        return dq

    def _set_ctrl(self, ctrl: np.ndarray) -> None:
        self.data.ctrl[: self.n_joints] = ctrl

    def _clip_to_ranges(self, q: np.ndarray) -> np.ndarray:
        out = q.copy()
        for i, js in enumerate(self.joints):
            out[i] = float(np.clip(out[i], js.range_min, js.range_max))
        return out

    def get_obs(self) -> np.ndarray:
        q = self._get_q()
        dq = self._get_dq()
        cube_pos = self.data.xpos[self.cube_body_id].copy()  # (3,)
        ee_pos = self.data.site_xpos[self.ee_site_id].copy()  # (3,)
        obs = np.concatenate([q, dq, cube_pos, ee_pos])
        return obs.astype(np.float32)

    # ---------------------------------------------------------------------
    # Public API (used by your Gym envs)
    # ---------------------------------------------------------------------
    def reset(self, seed: Optional[int] = None) -> np.ndarray:
        if seed is not None:
            self._rng = np.random.default_rng(seed)

        mujoco.mj_resetData(self.model, self.data)

        # --- Place cube on the table with small XY noise ---
        xy = self._cube_init_xy + self._rng.uniform(-self._cube_xy_noise, self._cube_xy_noise, size=(2,))
        x, y = float(xy[0]), float(xy[1])
        z = self._cube_init_z

        adr = self._cube_joint_qpos_adr
        # free joint: [x, y, z, qw, qx, qy, qz]
        self.data.qpos[adr : adr + 7] = np.array([x, y, z, 1.0, 0.0, 0.0, 0.0], dtype=np.float64)

        # --- Set robot to home pose ---
        for i, js in enumerate(self.joints):
            self.data.qpos[js.qpos_adr] = self._home_q[i]

        self.data.qvel[:] = 0.0

        # ctrl targets should match pose for a calm reset
        self._set_ctrl(self._clip_to_ranges(self._home_q))

        mujoco.mj_forward(self.model, self.data)
        return self.get_obs()

    def step(self, action: np.ndarray) -> np.ndarray:
        action = np.asarray(action, dtype=np.float64).reshape(self.n_joints)
        action = np.clip(action, -1.0, 1.0)

        # Delta position control around the current ctrl target (stable)
        cur_ctrl = self.data.ctrl[: self.n_joints].copy()
        target = cur_ctrl + self.action_scale * action
        target = self._clip_to_ranges(target)
        self._set_ctrl(target)

        for _ in range(self.ctrl_steps):
            mujoco.mj_step(self.model, self.data)

        return self.get_obs()
'''
with open("so_arm_simple_env.py", "w", encoding="utf-8") as f:
    f.write(so_arm_simple_env_py)

print("Wrote so_arm_simple_env.py")


Wrote so_arm_simple_env.py


In [4]:
# Overwrite so_arm_push_env.py with contact-gated LEFT-velocity reward + better success

so_arm_push_env_py = r'''import numpy as np
import gymnasium as gym
from gymnasium import spaces
import mujoco

from so_arm_simple_env import SoArmSimpleEnv


class SoArmPushEnv(gym.Env):
    """
    Push the cube to the LEFT goal.

    Obs (21): [q(6), dq(6), cube_pos(3), ee_pos(3), goal_pos(3)]
    Action (6): joint delta in [-1,1]
    """

    metadata = {"render_modes": ["human"], "render_fps": 60}

    def __init__(
        self,
        xml_path: str = "scene.xml",
        max_steps: int = 200,
        goal_delta_x: float = 0.10,
        goal_tolerance: float = 0.03,
    ):
        super().__init__()

        self.sim = SoArmSimpleEnv(
            xml_path=xml_path,
            control_hz=20,
            episode_seconds=max_steps / 20,
            action_scale=0.05,
        )

        self.max_steps = int(max_steps)
        self.step_count = 0

        self.goal_delta_x = float(goal_delta_x)
        self.goal_tolerance = float(goal_tolerance)

        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(self.sim.n_joints,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.sim.n_obs + 3,), dtype=np.float32)

        try:
            self.goal_site_id = self.sim.model.site("push_goal").id
        except KeyError:
            self.goal_site_id = None

        self.goal_pos = np.zeros(3, dtype=np.float32)
        self.prev_cube_x = 0.0
        self.prev_goal_dist = 0.0

    def _with_goal(self, base_obs: np.ndarray) -> np.ndarray:
        return np.concatenate([base_obs, self.goal_pos]).astype(np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        base_obs = self.sim.reset(seed=seed).astype(np.float32)

        n = self.sim.n_joints
        cube_pos = base_obs[n*2:n*2+3]

        # goal is left of initial cube x
        self.goal_pos = np.array([cube_pos[0] - self.goal_delta_x, cube_pos[1], cube_pos[2]], dtype=np.float32)

        # visualize goal marker
        if self.goal_site_id is not None:
            self.sim.model.site_pos[self.goal_site_id] = self.goal_pos
            mujoco.mj_forward(self.sim.model, self.sim.data)

        self.prev_cube_x = float(cube_pos[0])
        self.prev_goal_dist = float(np.linalg.norm(cube_pos - self.goal_pos))
        self.step_count = 0

        return self._with_goal(base_obs), {}

    def step(self, action):
        base_obs = self.sim.step(action).astype(np.float32)
        self.step_count += 1

        n = self.sim.n_joints
        cube_pos = base_obs[n*2:n*2+3]
        ee_pos   = base_obs[n*2+3:n*2+6]

        dist_ee_cube = float(np.linalg.norm(ee_pos - cube_pos))
        dist_cube_goal = float(np.linalg.norm(cube_pos - self.goal_pos))

        # cube moved left if x decreased
        cube_x = float(cube_pos[0])
        delta_x_left = self.prev_cube_x - cube_x
        self.prev_cube_x = cube_x

        # contact/near-cube gate (near=1, far->0)
        gate = float(np.exp(-12.0 * dist_ee_cube))

        # goal distance progress
        progress = self.prev_goal_dist - dist_cube_goal
        self.prev_goal_dist = dist_cube_goal

        # ---- Reward terms ----
        # (1) reach: learn to go to cube
        r_reach = 1.0 - float(np.tanh(6.0 * dist_ee_cube))

        # (2) push-left: ONLY if close to cube (gate)
        # make it strong so PPO prefers pushing instead of hovering
        r_push_left = 60.0 * delta_x_left * gate

        # (3) progress toward goal (also helpful)
        r_progress = 15.0 * progress

        # (4) final shaping
        r_goal = -1.5 * dist_cube_goal

        # (5) action smoothness
        a = np.asarray(action, dtype=np.float32)
        r_act = -0.01 * float(np.sum(np.abs(a)))

        reward = r_reach + r_push_left + r_progress + r_goal + r_act

        # success
        success = dist_cube_goal < self.goal_tolerance
        if success:
            reward += 80.0

        terminated = bool(success)
        truncated = self.step_count >= self.max_steps

        info = {
            "success": success,
            "dist_ee_cube": dist_ee_cube,
            "dist_cube_goal": dist_cube_goal,
            "delta_x_left": delta_x_left,
            "gate": gate,
        }

        return self._with_goal(base_obs), reward, terminated, truncated, info
'''

with open("so_arm_push_env.py", "w", encoding="utf-8") as f:
    f.write(so_arm_push_env_py)

print("✅ Updated so_arm_push_env.py (stronger push-left reward).")


✅ Updated so_arm_push_env.py (stronger push-left reward).


In [5]:
# Cell 5 — write train_ppo_push.py

train_ppo_push_py = r'''"""Train PPO to push the cube to the LEFT goal.

Run from the directory that contains:
  - scene.xml
  - so_arm100.xml
  - assets/ (mesh files)

Example:
  python3 train_ppo_push.py
  python3 visualize_ppo_push.py
"""

import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

from so_arm_push_env import SoArmPushEnv


def make_env():
    return SoArmPushEnv(xml_path="scene.xml", max_steps=200, goal_delta_x=0.10, goal_tolerance=0.03)


if __name__ == "__main__":
    vec_env = DummyVecEnv([make_env])

    model = PPO(
        policy="MlpPolicy",
        env=vec_env,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.0,
        vf_coef=0.5,
        max_grad_norm=0.5,
        verbose=1,
        tensorboard_log="./tb_push_ppo",
    )

    checkpoint = CheckpointCallback(save_freq=50_000, save_path="./checkpoints_push", name_prefix="ppo_push")

    total_timesteps = int(os.environ.get("TOTAL_TIMESTEPS", "1000000"))
    model.learn(total_timesteps=total_timesteps, callback=checkpoint)

    model.save("ppo_soarm_push")
    print("Saved: ppo_soarm_push.zip")

    vec_env.close()
'''
with open("train_ppo_push.py", "w", encoding="utf-8") as f:
    f.write(train_ppo_push_py)

print("Wrote train_ppo_push.py")


Wrote train_ppo_push.py


In [6]:
# Cell 6 — write visualize_ppo_push.py (opens mujoco.viewer and shows green goal marker)

visualize_ppo_push_py = r'''import time
import mujoco.viewer

from stable_baselines3 import PPO

from so_arm_push_env import SoArmPushEnv


if __name__ == "__main__":
    env = SoArmPushEnv(xml_path="scene.xml", max_steps=200)
    model = PPO.load("ppo_soarm_push")

    sim = env.sim
    obs, _ = env.reset()

    with mujoco.viewer.launch_passive(sim.model, sim.data) as viewer:
        done = False
        while viewer.is_running() and not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)

            if env.step_count % 20 == 0 or info.get("success", False):
                print(
                    f"t={env.step_count:3d} "
                    f"dist_goal={info['dist_cube_goal']:.4f} "
                    f"dist_ee={info['dist_ee_cube']:.4f} "
                    f"success={info['success']}"
                )

            done = terminated or truncated
            viewer.sync()
            time.sleep(sim.sim_dt * sim.ctrl_steps)

    env.close()
'''
with open("visualize_ppo_push.py", "w", encoding="utf-8") as f:
    f.write(visualize_ppo_push_py)

print("Wrote visualize_ppo_push.py")


Wrote visualize_ppo_push.py


In [7]:
# Cell 0 — install + train PPO + visualize (runs end-to-end from notebook)

import os, sys, subprocess, textwrap
from pathlib import Path

def run(cmd, env=None):
    print("\n>>>", " ".join(cmd))
    subprocess.check_call(cmd, env=env)

# -----------------------
# 1) Install dependencies
# -----------------------
run([sys.executable, "-m", "pip", "install", "-q", "stable-baselines3", "gymnasium", "mujoco"])

# -----------------------
# 2) Sanity checks
# -----------------------
required_files = [
    "scene.xml",
    "so_arm100.xml",
    "so_arm_simple_env.py",
    "so_arm_push_env.py",
    "train_ppo_push.py",
    "visualize_ppo_push.py",
]
missing = [f for f in required_files if not Path(f).exists()]
if missing:
    raise FileNotFoundError(
        "Missing files:\n"
        + "\n".join(f" - {m}" for m in missing)
        + "\n\nRun the earlier cells that write these files first."
    )

# MuJoCo meshes directory (must exist)
assets_dir = Path("assets")
if not assets_dir.exists():
    raise FileNotFoundError(
        "Missing 'assets/' folder with STL meshes.\n"
        "Place your mesh files in ./assets (same directory as scene.xml)."
    )

print("✅ All required files found.")

# -----------------------
# 3) Train PPO
# -----------------------
# Change these if you want:
TOTAL_TIMESTEPS = int(os.environ.get("TOTAL_TIMESTEPS", "1000000"))  # e.g. 300000, 1000000
env = os.environ.copy()
env["TOTAL_TIMESTEPS"] = str(TOTAL_TIMESTEPS)

print(f"\nTraining PPO with TOTAL_TIMESTEPS={TOTAL_TIMESTEPS} ...")
run([sys.executable, "train_ppo_push.py"], env=env)

# -----------------------
# 4) Visualize (MuJoCo viewer)
# -----------------------
print("\nLaunching viewer... You should see a GREEN sphere = goal position (push left).")
print("Close the viewer window to stop visualization.")
run([sys.executable, "visualize_ppo_push.py"], env=env)



>>> /bin/python -m pip install -q stable-baselines3 gymnasium mujoco
✅ All required files found.

Training PPO with TOTAL_TIMESTEPS=1000000 ...

>>> /bin/python train_ppo_push.py
Using cuda device


/home/salblooshi/.local/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Logging to ./tb_push_ppo/PPO_3
-----------------------------
| time/              |      |
|    fps             | 1169 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 970          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0101766605 |
|    clip_fraction        | 0.131        |
|    clip_range           | 0.2          |
|    entropy_loss         | -8.48        |
|    explained_variance   | 0.0838       |
|    learning_rate        | 0.0003       |
|    loss                 | 0.775        |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0163      |
|    std                  | 0.991        |
|    value_loss           | 

/home/salblooshi/.local/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


t= 18 dist_goal=0.0243 dist_ee=0.0287 success=True


CalledProcessError: Command '['/bin/python', 'visualize_ppo_push.py']' died with <Signals.SIGSEGV: 11>.

In [8]:
# Offscreen render rollout to MP4 (avoids mujoco.viewer SIGSEGV)

import numpy as np
import imageio.v2 as imageio
from stable_baselines3 import PPO
from so_arm_push_env import SoArmPushEnv
import mujoco

model = PPO.load("ppo_soarm_push")
env = SoArmPushEnv(xml_path="scene.xml", max_steps=200)
sim = env.sim

# Offscreen renderer
renderer = mujoco.Renderer(sim.model, height=480, width=640)

obs, _ = env.reset()
frames = []

for t in range(200):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

    renderer.update_scene(sim.data)
    frame = renderer.render()  # RGB uint8
    frames.append(frame)

    if terminated or truncated:
        break

out_path = "push_rollout.mp4"
imageio.mimsave(out_path, frames, fps=20)
print("Saved video:", out_path)


/home/salblooshi/.local/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Saved video: push_rollout.mp4
